# Four panel model

Introduction:
------------------

Four-Panel Model (Four panels per grid cell, 4 orientations): In this model,for each time-step the power output is calculated from four tilted PV panels per grid cell. They are oriented towards North, East, South and West with a user-selectable and optimizable slope. The model separately considers flat and pitched roofs. For the flat roofs, PV panels are considered South facing for power output estimation with optimised slope. For pitched roofs, the summed PV power output of the four panels (equally oriented in four cardinal directions) is used. The PV power output from flat and pitched roofs are scaled using the respective projected roof area and totalled over the grid cell to calculate a total idealized PV power output. Finally the output is reduced due to effects of geometric placement and shading.

Methodology:
------------------
This model was developed using LiDAR and DSM data. Forty simulation areas have been generated in which 25 training areas are used to analyze the relationships between PV power production potential and urban forms.

Later parametrized equations, namely placement efficiency and shading efficiency are generated from the training areas, which were later evaluated using the evaluation areas. Placement efficiency was used to accurately calculate geometrically placed PV panels for an urban area without actually placing PV panels on rooftop building segments. Shading efficiency calculates the PV power output with shading losses instead of using the SEBE model. With the placement and shading efficiency, PV power output for an urban area can be calculated in a simple and fast manner.

Significance:
------------------
1. With fewer input parameters, can rapidly estimate the PV potential over an entire year.
2. Parametrized equations used in this model assist in calculating geometrically placed PV panels and account for shading losses.

Limitations:
-----------------
1. Assumes all rooftop building segments are equipped with PV panels neglecting safety, socio-economic and environmental factors.

Output:
-----------

Hourly PV power output [W/m2] for an urban area

In [9]:
    """

    Input Parameters
    ----------
        l            : Length of PV panel in portait mode [m] 
        w            : Width of PV panel in portait mode [m]
        pv_eff       : Efficiency of PV panel [%]
        alpha        : Temperature coefficient of PV panel [1/K]
        lamda_p      : Projected plan area fraction of building segments
        l_s          : Average building segment length [m] is the typical distance of the length of the roof, 
                       which tells you how far the shadows get on the roof and is calculated 
                       by taking the square root of the average segment area.
        f_f          : Fraction of flat building segments
        wire_losses  : Wiring losses
        f_c          : Capacity curve coefficient(f_c = 1 if all the rooftop segments are covered with panels)
        d            : Panel row spacing, can be optimized according to technical & Economical parameters
                       In this paper, d = 1.5 m
        β_o          : Optimal slope of PV panels on flat rooftops, can be optimized according to location
                       In this paper, β_0 = 53°(Freier et al., 2018) for the city of Berlin.
        albedo       : System albedo for an urban area            
        urban_area   : Area of the study area/ urban area [m2]
        metadata     : Input meteorological data specifically formatted using 
                       UMEP -> Pre-processing -> Meteorological data -> Prepare existing data
                       Here, a sample Random Meteorological Day(RMD) for the month of June has been given as an example.
                       Annual weather data for the city of Berlin has been attached for reference.
        sol_angles   : Sun position angles for the urban area.
                       Here, solar angles for the RMD - June has been given as example
        
        Note: 1. In this script, l, w, pv_eff, alpha values has been taken for a standard PV module 
                 but the user can change these parameters according to their module.
              2. lamda_p, l_s, f_f and urban_area values has been taken for an random urban area in the city of Berlin
              3. Input files and parameters can be found under the folder "Datasets" in Github

    """

import numpy as np
import pandas as pd
import math
import pvlib 

l           = 1
w           = 1.6  
pv_eff      = 0.177 
alpha       = -0.0039  
lamda_p     = 0.398  
l_s         = 9.36  
f_f         = 0.413  
wire_losses = 0.8
f_c         = 1
d           = 1.5
β_o         = 40
albedo      = 0.2
urban_area  = 265200
metdata     = np.loadtxt("june_RMD.txt", skiprows=1, dtype=float).reshape((-1, 24))
sol_angles  = pd.read_csv(f'june_angles_RMD.csv',index_col=0)

In [10]:
  """
    Constants
    ---------
        ref_temp     : Reference PV cell temperature, default value = 25 °C
        k_t          : Reduction factor, default value = 0.05 °C/(W/m2)
            
            Reduction factor (kT) a constant defined as “expressing the changes in module performance due to 
            differences in the module’s actual , and nominal operating temperatures” (Ramirez Camargo et al., 2015). 
            A constant value for kT = 0.05 °C/(W/m2) was suggested by various authors (King et al., 2004), 
            (Ramirez Camargo et al., 2015) and also used in the PV-GIS web service for 
            building integrated systems.
            
            References:
            ----------
            
            Ramirez Camargo, L., Zink, R., Dorner, W., & Stoeglehner, G. (2015). 
            Spatio-temporal modelling of roof-top photovoltaic panels for improved technical potential assessment 
            and electricity peak load offsetting at the municipal scale. Computers, Environment, and Urban Systems, 
            52, 58–69. https://doi.org/10.1016/j.compenvurbsys.2015.03.002
            
            King, D. L., Boyson, W. E., & Kratochvill, J. A. (2004). Photovoltaic Array Performance Model. December.
            
            
  """

ref_temp = 25
k_t      = 0.05

In [11]:
  """
    Theoretical PV panels
    ---------------------
    
        PV panels are calculated using rooftop building segment area without considering the geometry, tilt, 
        and aspect of the building segment.
        
  """
panel_area      = l * w
theoretical_PVP = (f_f * lamda_p / ((panel_area * math.cos(math.radians(β_o))) + d)) + ((1 - f_f) * lamda_p / (panel_area * math.cos(math.radians(β_o))))

In [12]:
  """
    Parametrized Equations
    ----------------------
    
        pl_eff       : Placement efficiency is the ratio of geometrically installed PV panels
                       to the theoretical PV panels.
                
        dir_eff      : Direct shading efficiency loss factor is the reduction of direct POA irradiance due to shading 
                       and is estimated by comparing the direct POA irradiance for Models 3 with building resolving model
                       
        dif_eff      : Diffuse shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the diffuse POA irradiance for Models 3 with building resolving model
        
        ref_eff      : Reflected shading efficiency loss factor is the reduction of diffuse POA irradiance due to shading 
                       and is estimated by comparing the reflected POA irradiance for Models 3 with building resolving model
                
  """
    
pl_eff  = (-0.0968 * f_f + 0.3232)
dir_eff = (-0.00039 * l_s - 0.00533)
dif_eff = (-0.00637 * l_s + 0.36056)
ref_eff = (0.00799 * l_s + 0.60976)

In [13]:
  """
    Geometric PV panels
    -------------------
        
        Geometrical distribution of PV panels on rooftop building segments are calculated 
        using Theoretical PV panels and placement efficiency

  """
geometric_PVP = theoretical_PVP * (1 - pl_eff)

# Dividing the panels into flat and pitched and panels on pitched roofs are equally divided into four orientations.
# 0 = North, 90 - East, 180 - South, 270 - West 

panels = {
180     :   geometric_PVP * f_f, # panels over flat rooftop segments
0       :   geometric_PVP * (1 - f_f) * 0.25,
90      :   geometric_PVP * (1 - f_f) * 0.25,
180.01  :   geometric_PVP * (1 - f_f) * 0.25, # 180.01 is taken for differentiating from flat panels @ 180
270     :   geometric_PVP * (1 - f_f) * 0.25
}

In [14]:
pv_power = pd.DataFrame()

def four_panel(slope, panels, aspect):


    for i in range(metdata.shape[0]):

        if metdata[[i], 14] > 0:

            Glob_hor = metdata[[i], 14]
            Diff_hor = metdata[[i], 21]
            Dir_hor  = metdata[[i], 22]
            phi      = sol_angles.loc[int(metdata[[i], 2]), 'azi']
            gamma    = sol_angles.loc[int(metdata[[i], 2]), 'gamma']
            zenith   = sol_angles.loc[int(metdata[[i], 2]), 'zen']
            aoi      = pvlib.irradiance.aoi(slope, aspect, zenith, phi)
            metdata1 = metdata[[i], :]
            hour = int(metdata1[0,2])

            if gamma > 0 :

                lamda = math.cos(math.radians(phi - aspect)) + (math.tan(math.radians(gamma)) / math.tan(math.radians(slope)))

                if lamda <= 0 :

                    f_s1 = 1
                    f_s2 = 1

                else:

                    f_s1 = 0
                    x_l  = (math.cos(math.radians(slope)) + d/l) * math.sin(math.radians(phi - aspect)) / lamda
                    y_l  = 1 - ((math.cos(math.radians(slope)) + d/l) * math.tan(math.radians(gamma)) / (lamda * math.sin(math.radians(slope))))

                    if y_l <=0:

                        f_s2 = 0

                    elif abs(x_l) > (w / l):

                        f_s2 = 0

                    else:

                        f_s2 = (1 - (abs(x_l)/ (w / l))) * y_l

                f_sky1 = (1 + math.cos(math.radians(slope))) / 2

                B       = Dir_hor * (math.cos(math.radians(aoi)) / math.sin(math.radians(gamma)))                
                poa_dir = (1 - f_s1) * B * (1 - (dir_eff * gamma + 1))
                poa_dif = f_sky1 * Diff_hor * (1 - dif_eff)
                ref_dif = albedo * Glob_hor * (1 - f_sky1) * (1 - ref_eff)
                
                pv_power.loc[i+1,f'Year']      = int(metdata1[0,0])
                pv_power.loc[i+1,f'Day']       = int(metdata1[0,1])
                pv_power.loc[i+1,f'Hour']      = hour
                pv_power.loc[i+1,f'dir_{aspect}'] = poa_dir * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dir) - ref_temp)) * panel_area * panels * wire_losses * f_c
                pv_power.loc[i+1,f'dif_{aspect}'] = poa_dif * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * poa_dif) - ref_temp)) * panel_area * panels * wire_losses * f_c
                pv_power.loc[i+1,f'ref_{aspect}'] = ref_dif * pv_eff * (1 + alpha * ((metdata[[i], 11] + k_t * ref_dif) - ref_temp)) * panel_area * panels * wire_losses * f_c


for aspect, panel in panels.items():
        
    four_panel(β_o, panel, aspect)

In [15]:
pv_power['direct']    = pv_power['dir_180'] + pv_power['dir_0'] + pv_power['dir_90']+ pv_power['dir_180.01']+ pv_power['dir_270']
pv_power['diffuse']   = pv_power['dif_180'] + pv_power['dif_0'] + pv_power['dif_90']+ pv_power['dif_180.01']+ pv_power['dif_270']
pv_power['reflected'] = pv_power['ref_180'] + pv_power['ref_0'] + pv_power['ref_90']+ pv_power['ref_180.01']+ pv_power['ref_270']

pv_power['Total_PVP'] = (pv_power['direct'] + pv_power['diffuse'] + pv_power['reflected']) * urban_area / 1000000

In [16]:
pv_power

,Year,Day,Hour,dir_180,dif_180,ref_180,dir_0,dif_0,ref_0,dir_90,...,dir_180.01,dif_180.01,ref_180.01,dir_270,dif_270,ref_270,direct,diffuse,reflected,Total_PVP
5,2019.0,167.0,5.0,0.000000,0.279779,0.005290,0.037202,0.099413,0.001880,0.037330,...,0.000000,0.099413,0.001880,0.000000,0.099413,0.001880,0.074531,0.677430,0.012810,0.202817
6,2019.0,171.0,6.0,0.000000,0.599605,0.018103,0.208010,0.213056,0.006433,0.209822,...,0.000000,0.213056,0.006433,0.000000,0.213056,0.006433,0.417833,1.451827,0.043833,0.507458
7,2019.0,160.0,7.0,1.285067,1.010222,0.036368,0.461130,0.358959,0.012922,0.466987,...,0.456617,0.358959,0.012922,0.000000,0.358959,0.012922,2.669801,2.446059,0.088057,1.380079
8,2019.0,152.0,8.0,1.309396,1.391266,0.040975,0.466136,0.494354,0.014560,0.473228,...,0.465262,0.494354,0.014560,0.000000,0.494354,0.014560,2.714022,3.368684,0.099214,1.639445
9,2019.0,175.0,9.0,3.178827,1.192892,0.072380,1.125939,0.423867,0.025719,1.142809,...,1.129520,0.423867,0.025719,0.000000,0.423867,0.025719,6.577095,2.888358,0.175255,2.556716
10,2019.0,179.0,10.0,0.436106,2.660033,0.040173,0.153697,0.945181,0.014274,0.156030,...,0.154960,0.945181,0.014274,0.152627,0.945181,0.014274,1.053420,6.440758,0.097271,2.013252
11,2019.0,169.0,11.0,4.482364,1.452454,0.094286,1.575062,0.516096,0.033502,1.595290,...,1.592702,0.516096,0.033502,1.572472,0.516096,0.033502,10.817891,3.516837,0.228294,3.862114
12,2019.0,174.0,12.0,5.015066,1.464122,0.100955,1.758893,0.520242,0.035872,1.777375,...,1.781987,0.520242,0.035872,1.763514,0.520242,0.035872,12.096834,3.545089,0.244443,4.213064
13,2019.0,156.0,13.0,4.253471,1.819040,0.091343,1.489767,0.646354,0.032456,1.501031,...,1.511372,0.646354,0.032456,1.500119,0.646354,0.032456,10.255761,4.404456,0.221169,3.946543
14,2019.0,165.0,14.0,4.754097,1.401464,0.095242,1.666639,0.497978,0.033842,1.672870,...,1.689259,0.497978,0.033842,1.683036,0.497978,0.033842,11.465901,3.393376,0.230609,4.001838
